# Easy Dev for Post-hoc OOD Detectors

This notebook integrates some simple post-hoc OOD detection methods.

We choose ImageNet-1K as in-distribution (ID) and load a pretrained vision transformer (ViT).

In [1]:
%load_ext autoreload
%autoreload 2

## Load Models and Dataset

In [2]:
from openood.utils import config
from openood.datasets import get_dataloader, get_ood_dataloader
from openood.evaluators import get_evaluator
from openood.networks import get_network
from torchvision.transforms import RandAugment
from torchvision import transforms

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/gridsan/nhulkund/.local/lib/python3.8/site-packages/mmcv/cnn/bricks/transformer.py:32: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


In [3]:
# # load config files for cifar10 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/cifar10/cifar10.yml',
#     './configs/datasets/cifar10/cifar10_ood.yml',
#     './configs/networks/resnet18_32x32.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/msp.yml',
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar10_res18_acc94.30.ckpt'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()

# load config files for cifar100 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/cifar100/cifar100.yml',
#     './configs/datasets/cifar100/cifar100_ood.yml',
#     './configs/networks/resnet18_32x32.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/msp.yml',
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar100_res18_acc78.20.ckpt'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()
# load config files for cifar10 baseline
PATH="/home/gridsan/nhulkund/OpenOOD/"
config_files = [
    './configs/datasets/imagenet/imagenet.yml',
    './configs/datasets/imagenet/imagenet_ood.yml',
    './configs/networks/resnet50.yml',
    './configs/pipelines/test/test_ood.yml',
    './configs/preprocessors/base_preprocessor.yml',
    './configs/postprocessors/msp.yml',
]
config = config.Config(*config_files)
# modify config 
#config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/imagenet_res50_acc76.10.pth'
config.network.pretrained = True
config.num_workers = 8
config.save_output = False
config.parse_refs()

In [4]:
config.network

checkpoint: /home/gridsan/nhulkund/OpenOOD//scripts/download/results/checkpoints/imagenet_res50_acc76.10.pth
image_size: 224
name: resnet50
num_classes: 1000
num_gpus: 1
pretrained: True

In [2]:
#print(config)

In [6]:
# get dataloader
id_loader_dict = get_dataloader(config)
ood_loader_dict = get_ood_dataloader(config)
# init network
net = get_network(config.network).cuda()
# init ood evaluator
evaluator = get_evaluator(config)

Model Loading resnet50 Completed!


## Feature Extraction

In [7]:
from tqdm import tqdm
import numpy as np
import torch
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from skimage import img_as_ubyte
from PIL import Image



In [8]:

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [9]:
def save_arr_to_dir(arr, dir):
    os.makedirs(os.path.dirname(dir), exist_ok=True)
    with open(dir, 'wb+') as f:
        np.save(f, arr)

In [10]:
save_root = PATH+f'/results/{config.exp_name}'


In [11]:
# save id (test & val) results
net.eval()
modes = ['test', 'val']
for mode in modes:
    dl = id_loader_dict[mode]
    dataiter = iter(dl)
    
    logits_list = []
    feature_list = []
    label_list = []
    id_list = []
    count=0
    print(id_list)
    for i in tqdm(range(1,
                    len(dataiter) + 1),
                    desc='Extracting reults...',
                    position=0,
                    leave=True):
        batch = next(dataiter)
        data = batch['data'].cuda()
        label = batch['label']
        with torch.no_grad():
            logits_cls, feature = net(data, return_feature=True)
        logits_list.append(logits_cls.data.to('cpu').numpy())
        feature_list.append(feature.data.to('cpu').numpy())
        label_list.append(label.numpy())

    logits_arr = np.concatenate(logits_list)
    feature_arr = np.concatenate(feature_list)
    label_arr = np.concatenate(label_list)
    
    save_arr_to_dir(logits_arr, osp.join(save_root, 'id', f'{mode}_logits.npy'))
    save_arr_to_dir(feature_arr, osp.join(save_root, 'id', f'{mode}_feature.npy'))
    save_arr_to_dir(label_arr, osp.join(save_root, 'id', f'{mode}_labels.npy'))

[]


Extracting reults...: 100%|██████████| 1407/1407 [01:03<00:00, 22.30it/s]


[]


Extracting reults...: 100%|██████████| 157/157 [00:07<00:00, 22.24it/s]


In [12]:
# save ood results
net.eval()
ood_splits = ['nearood', 'farood']
for ood_split in ood_splits:
    for dataset_name, ood_dl in ood_loader_dict[ood_split].items():
        dataiter = iter(ood_dl)
    
        logits_list = []
        feature_list = []
        label_list = []

        for i in tqdm(range(1,
                        len(dataiter) + 1),
                        desc='Extracting reults...',
                        position=0,
                        leave=True):
            batch = next(dataiter)
            data = batch['data'].cuda()
            label = batch['label']

            with torch.no_grad():
                logits_cls, feature = net(data, return_feature=True)
            logits_list.append(logits_cls.data.to('cpu').numpy())
            feature_list.append(feature.data.to('cpu').numpy())
            label_list.append(label.numpy())

        logits_arr = np.concatenate(logits_list)
        feature_arr = np.concatenate(feature_list)
        label_arr = np.concatenate(label_list)

        save_arr_to_dir(logits_arr, osp.join(save_root, ood_split, f'{dataset_name}_logits.npy'))
        save_arr_to_dir(feature_arr, osp.join(save_root, ood_split, f'{dataset_name}_feature.npy'))
        save_arr_to_dir(label_arr, osp.join(save_root, ood_split, f'{dataset_name}_labels.npy'))

Extracting reults...: 100%|██████████| 282/282 [00:12<00:00, 22.71it/s]


## MSP Evaluation

In [13]:
# build msp method (pass in pre-saved logits)
def msp_postprocess(logits):
    score = torch.softmax(logits, dim=1)
    conf, pred = torch.max(score, dim=1)
    return pred, conf

def msp_aug_postprocess(logits):
    score = torch.softmax(logits, dim=1)
    if score.shape[0] % 11 == 0: 
        first_dim=int(score.shape[0]/11)
        second_dim=11
    else:
        print(score.shape)
    score_reshaped=score.numpy().reshape(first_dim,second_dim,10)
    score_averaged=torch.Tensor(np.mean(score_reshaped,axis=1))
    conf, pred = torch.max(score_averaged, dim=1)
    return pred, conf

In [14]:
# load logits, feature, label for this benchmark
results = dict()
# for id
modes = ['val', 'test']
results['id'] = dict()
for mode in modes:
    results['id'][mode] = dict()
    results['id'][mode]['feature'] = np.load(osp.join(save_root, 'id', f'{mode}_feature.npy'))
    results['id'][mode]['logits'] = np.load(osp.join(save_root, 'id', f'{mode}_logits.npy'))
    results['id'][mode]['labels'] = np.load(osp.join(save_root, 'id', f'{mode}_labels.npy'))

# for ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        results[split_type][dataset_name] = dict()
        results[split_type][dataset_name]['feature'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_feature.npy'))
        results[split_type][dataset_name]['logits'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_logits.npy'))
        results[split_type][dataset_name]['labels'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_labels.npy'))
        


In [16]:
# print(results['nearood']['cifar10']['logits'].shape)

In [17]:
def print_nested_dict(dict_obj, indent = 0):
    ''' Pretty Print nested dictionary with given indent level  
    '''
    # Iterate over all key-value pairs of dictionary
    for key, value in dict_obj.items():
        # If value is dict type, then print nested dict 
        if isinstance(value, dict):
            print(' ' * indent, key, ':', '{')
            print_nested_dict(value, indent + 2)
            print(' ' * indent, '}')
        else:
            print(' ' * indent, key, ':', value.shape)


In [18]:
print_nested_dict(results)

 id : {
   val : {
     feature : (5000, 2048)
     logits : (5000, 1000)
     labels : (5000,)
   }
   test : {
     feature : (45000, 2048)
     logits : (45000, 1000)
     labels : (45000,)
   }
 }
 nearood : {
   species : {
     feature : (10000, 2048)
     logits : (10000, 1000)
     labels : (10000,)
   }
   inaturalist : {
     feature : (10000, 2048)
     logits : (10000, 1000)
     labels : (10000,)
   }
   openimageo : {
     feature : (15869, 2048)
     logits : (15869, 1000)
     labels : (15869,)
   }
   imageneto : {
     feature : (2000, 2048)
     logits : (2000, 1000)
     labels : (2000,)
   }
 }
 farood : {
   texture : {
     feature : (5160, 2048)
     logits : (5160, 1000)
     labels : (5160,)
   }
   mnist : {
     feature : (9000, 2048)
     logits : (9000, 1000)
     labels : (9000,)
   }
 }


In [19]:
# get pred, conf, gt from MSP postprocessor (can change to your custom_postprocessor here)
postprocess_results = dict()
# id
modes = ['val', 'test']
postprocess_results['id'] = dict()
for mode in modes:
    pred, conf = msp_postprocess(torch.from_numpy(results['id'][mode]['logits']))
    pred, conf = pred.numpy(), conf.numpy()
    gt = results['id'][mode]['labels']
    postprocess_results['id'][mode] = [pred, conf, gt]

# ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    postprocess_results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        pred, conf = msp_postprocess(torch.from_numpy(results[split_type][dataset_name]['logits']))
        pred, conf = pred.numpy(), conf.numpy()
        gt = results[split_type][dataset_name]['labels']
        gt = -1 * np.ones_like(gt)   # hard set to -1 here
        postprocess_results[split_type][dataset_name] = [pred, conf, gt]

In [20]:
def print_all_metrics(metrics):
    [fpr, auroc, aupr_in, aupr_out,
        ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
        = metrics
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
            end=' ',
            flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
            flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
            end=' ',
            flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)   

In [21]:
from openood.evaluators.metrics import compute_all_metrics
def eval_ood(postprocess_results):
    [id_pred, id_conf, id_gt] = postprocess_results['id']['test']
    split_types = ['nearood', 'farood']

    for split_type in split_types:
        metrics_list = []
        print(f"Performing evaluation on {split_type} datasets...")
        dataset_names = config['ood_dataset'][split_type].datasets
        
        for dataset_name in dataset_names:
            [ood_pred, ood_conf, ood_gt] = postprocess_results[split_type][dataset_name]

            pred = np.concatenate([id_pred, ood_pred])
            conf = np.concatenate([id_conf, ood_conf])
            label = np.concatenate([id_gt, ood_gt])
            print(f'Computing metrics on {dataset_name} dataset...')

            ood_metrics = compute_all_metrics(conf, label, pred)
            print_all_metrics(ood_metrics)
            metrics_list.append(ood_metrics)
        print('Computing mean metrics...', flush=True)
        metrics_list = np.array(metrics_list)
        metrics_mean = np.mean(metrics_list, axis=0)   
        print_all_metrics(metrics_mean)

 

In [22]:
print("THIS IS NON RAND AUGMENT")
eval_ood(postprocess_results)

THIS IS NON RAND AUGMENT
Performing evaluation on nearood datasets...
Computing metrics on species dataset...
FPR@95: 79.72, AUROC: 75.18 AUPR_IN: 92.92, AUPR_OUT: 38.36
CCR: 0.75, 2.62, 12.65, 40.16, ACC: 76.18
──────────────────────────────────────────────────────────────────────
Computing metrics on inaturalist dataset...
FPR@95: 52.77, AUROC: 88.41 AUPR_IN: 97.09, AUPR_OUT: 63.88
CCR: 8.11, 18.64, 35.58, 61.28, ACC: 76.18
──────────────────────────────────────────────────────────────────────
Computing metrics on openimageo dataset...
FPR@95: 64.04, AUROC: 84.86 AUPR_IN: 94.11, AUPR_OUT: 64.60
CCR: 2.55, 8.76, 29.61, 56.44, ACC: 76.18
──────────────────────────────────────────────────────────────────────
Computing metrics on imageneto dataset...
FPR@95: 100.00, AUROC: 28.60 AUPR_IN: 94.30, AUPR_OUT: 2.79
CCR: 0.00, 1.19, 3.57, 12.36, ACC: 76.18
──────────────────────────────────────────────────────────────────────
Computing mean metrics...
FPR@95: 74.13, AUROC: 69.26 AUPR_IN: 94.61,